In [ ]:
import pandas as pd
import os
import re
import json
import yaml
from glob import glob
from tqdm import tqdm
from pprint import pprint
import torch
import pytorch_lightning as pl
from rouge import Rouge # 모델의 성능을 평가하기 위한 라이브러리입니다.

from torch.utils.data import Dataset , DataLoader
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, BartForConditionalGeneration, BartConfig
from transformers import T5TokenizerFast, T5ForConditionalGeneration, T5Config
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback

import wandb # 모델 학습 과정을 손쉽게 Tracking하고, 시각화할 수 있는 라이브러리입니다.

# visualization
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/NanumFont/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# config 설정에 tokenizer 모듈이 사용되므로 미리 tokenizer를 정의해줍니다.
model_dir = "lcw99/t5-base-korean-text-summary"
tokenizer = T5TokenizerFast.from_pretrained(model_dir) 

# https://huggingface.co/paust/pko-t5-base
# transformers 의 API 를 사용하여 접근 가능합니다. 
# tokenizer 를 사용할때는 T5Tokenizer 가 아니라 T5TokenizerFast 를 사용해주십시오. 
# model 은 T5ForConditionalGeneration 를 그대로 활용하시면 됩니다.

In [ ]:
config_data = {
    "general": {
        "data_path": "your_path", # 모델 생성에 필요한 데이터 경로를 사용자 환경에 맞게 지정합니다.
        "model_name": f"{model_dir}", # 불러올 모델의 이름을 사용자 환경에 맞게 지정할 수 있습니다.
        "output_dir": "./" # 모델의 최종 출력 값을 저장할 경로를 설정합니다.
    },
    "tokenizer": {
        "encoder_max_len": 1024,
        "decoder_max_len": 150,
        "bos_token": f"{tokenizer.bos_token}",
        "eos_token": f"{tokenizer.eos_token}",
        "unk_token": f"{tokenizer.unk_token}",
        "pad_token": f"{tokenizer.pad_token}",
        # 특정 단어들이 분해되어 tokenization이 수행되지 않도록 special_tokens을 지정해줍니다.
        "special_tokens": ['#Person1#', '#Person2#', '#Person3#', '#Person4#', '#Person5#', '#Person6#', '#Person7#',
                           '#DateOfBirth#', '#CarNumber#', '#Email#', '#CardNumber#', '#Address#', '#SSN#', '#PhoneNumber#', '#PassportNumber#']
    },
    "training": {
        "overwrite_output_dir": True,
        "num_train_epochs": 50,
        "learning_rate": 1e-5,
        "per_device_train_batch_size": 4, # 50
        "per_device_eval_batch_size": 16, # 32
        "warmup_ratio": 0.1,
        "weight_decay": 0.01,
        "lr_scheduler_type": 'cosine',
        "optim": 'adamw_torch',
        "gradient_accumulation_steps": 16, # 몇 개의 작은 배치를 합쳐서 큰 배치처럼 처리할지를 결정하는 파라미터: 메모리 제한 극복, 더 나은 성능
        "evaluation_strategy": 'epoch',
        "save_strategy": 'epoch',
        "save_total_limit": 7,
        "fp16": True,
        "load_best_model_at_end": True,
        "seed": 42,
        "logging_dir": "./logs",
        "logging_strategy": "epoch",
        "predict_with_generate": True,
        "generation_max_length": 150,
        "do_train": True,
        "do_eval": True,
        "early_stopping_patience": 5, # 3
        "early_stopping_threshold": 0.0001, # 0.001
        "report_to": "wandb" # (선택) wandb를 사용할 때 설정합니다.
    },
    # (선택) wandb 홈페이지에 가입하여 얻은 정보를 기반으로 작성합니다.
    "wandb": {
        "entity": "entity_name",
        "project": "project_name",
        "name": "name"
    },
    "inference": {
        "ckt_path": "model ckt path", # 사전 학습이 진행된 모델의 checkpoint를 저장할 경로를 설정합니다.
        "result_path": "./prediction/",
        "no_repeat_ngram_size": 2,
        "early_stopping": True,
        "generate_max_length": 150,
        "num_beams": 4,
        "batch_size" : 16, # 32
        # 정확한 모델 평가를 위해 제거할 불필요한 생성 토큰들을 정의합니다.
        "remove_tokens": ['<usr>', f"{tokenizer.bos_token}", f"{tokenizer.eos_token}", f"{tokenizer.unk_token}", f"{tokenizer.pad_token}"]
    }
}

In [ ]:
# 모델의 구성 정보를 YAML 파일로 저장합니다.
config_path = "./config.yaml"
with open(config_path, "w") as file:
    yaml.dump(config_data, file, allow_unicode=True)

# 저장된 config 파일을 불러옵니다.
config_path = "./config.yaml"

with open(config_path, "r") as file:
    loaded_config = yaml.safe_load(file)

loaded_config['general']['data_path'] = '../../data'

In [ ]:
# config에 저장된 데이터 경로를 통해 train과 validation data를 불러옵니다.
data_path = loaded_config['general']['data_path']

# train data의 구조와 내용을 확인합니다.
train_df = pd.read_csv(os.path.join(data_path,'train.csv'))
display(train_df.tail())

# dev data의 구조와 내용을 확인합니다.
dev_df = pd.read_csv(os.path.join(data_path,'dev.csv'))
display(dev_df.tail())

train_df_copy = train_df.copy()
dev_df_copy = dev_df.copy()

# 대화문에 개행문자 적용 잘 되어있는지 확인

In [ ]:
train_dialogue_n = train_df_copy['dialogue'].apply(lambda x: x.count('\n'))
train_dialogue_n.describe()

In [ ]:
n_1_list = train_dialogue_n[train_dialogue_n <= 1].reset_index()['index'].tolist()
for idx in n_1_list:
    print(f'\n[{idx}]')
    print('-----'*20)
    print(train_df_copy.iloc[idx, 1])
    print('-----'*20)

# special token을 통해 적절하지 않은 형식 찾기

> 데이터 전처리에서 얻은 special token

{'#Person3#', '#PhoneNumber#', '#Person5#', '#PassportNumber#', '#DateOfBirth#', '#Address#', '#Person6#', '#CarNumber#', '#SSN#', '#Person7#', '#CardNumber#', '#Person4#', '#Person2#', '#Person#', '#Email#', '#Person1#'}

In [ ]:
def findall_pattern(df, pattern):
    idx_list = []
    for idx in range(0, df.shape[0]):
        text = df.loc[idx, 'dialogue']
        pattern = pattern
        masked = re.findall(pattern, text)
        if len(masked) > 0:
            idx_list.append(idx)
    return idx_list, masked

def fullmatch_pattern(df, pattern):
    idx_list = []
    for idx in range(0, df.shape[0]):
        text = df.loc[idx, 'dialogue']
        pattern = pattern
        masked = re.fullmatch(pattern, text)
        if len(masked) > 0:
            idx_list.append(idx)
    return idx_list, masked

## Train

In [ ]:
train_df_person = train_df.copy()
del_token = ['#PhoneNumber#', '#PassportNumber#', '#DateOfBirth#', '#Address#', '#CarNumber#', '#SSN#', '#CardNumber#', '#Email#']
pattern = '|'.join(re.escape(token) for token in del_token)
for idx in tqdm(range(0, train_df_copy.shape[0])):
    text = train_df_copy.iloc[idx, 1]
    text2 = re.sub(pattern, '', text)
    train_df_person.iloc[idx, 1] = text2

In [ ]:
print(train_df_person.iloc[12428, 1])

### Person이 아니라 한국어인 경우

In [ ]:
idx_list, _ = findall_pattern(train_df_person, r'#[가-힣]+')
for idx in idx_list:
    print(f'\n[{idx}]')
    print('----'*20)
    print(train_df_person.iloc[idx, 1])
    print('----'*20)
print(len(idx_list), idx_list)

### 영어와 한국어이면서, 마지막이 숫자가 없거나 #으로 안 끝나는 경우

In [ ]:
idx_list2, _ = findall_pattern(train_df_person, r'#\b[A-Za-z가-힣]+\b')
for idx in idx_list2:
    print(f'\n[{idx}]')
    print('----'*20)
    print(train_df_person.iloc[idx, 1])
    print('----'*20)
print(len(idx_list2), idx_list2)

### 첫시작에 #이 없는 경우

In [ ]:
idx_list3, _ = findall_pattern(train_df_person, r'(?<!#)\b[A-Za-z가-힣0-9]+\b#')
for idx in idx_list3:
    print(f'\n[{idx}]')
    print('----'*20)
    print(train_df_person.iloc[idx, 1])
    print('----'*20)
print(len(idx_list3))

### 양쪽에 #이 없는 경우

In [ ]:
idx_list4, _ = findall_pattern(train_df_person, r'(?<!#)\b사람[0-9]+:\s\b') # 사람 Person
for idx in idx_list4:
    print(f'\n[{idx}]')
    print('----'*20)
    print(train_df_person.iloc[idx, 1])
    print('----'*20)
print(len(idx_list4))

### 4가지 경우의 인덱스 값들 중복 제거

In [ ]:
person_idx = list(set(idx_list + idx_list2 + idx_list3 + idx_list4))
person_idx = sorted(person_idx)
print(len(person_idx), person_idx)

In [ ]:
for idx in person_idx:
    print(f'\n[{idx}]')
    print('----'*20)
    print(train_df_copy.iloc[idx, 1])
    print('----'*20)

In [ ]:
print(
    re.sub(r'#PhoneNumber[가-힣]', '#PhoneNumber#', train_df_copy.iloc[420, 1])
)

In [ ]:
train_df_copy.iloc[420, 1] = re.sub(r'#PhoneNumber[가-힣]', '#PhoneNumber#', train_df_copy.iloc[420, 1])
train_df_copy.iloc[839, 1] = re.sub(r'#사람1', '#Person1#: ', train_df_copy.iloc[839, 1])
train_df_copy.iloc[1125, 1] = re.sub(r'사람1#', '#Person1#', train_df_copy.iloc[1125, 1])

train_df_copy.iloc[1142, 1] = re.sub(r'사람1#', '#Person1#', train_df_copy.iloc[1142, 1])
train_df_copy.iloc[1213, 1] = re.sub(r'#하지만', '#Person1#:', train_df_copy.iloc[1213, 1])
train_df_copy.iloc[1236, 1] = re.sub(r'#고객님', '#Person2#: 고객님', train_df_copy.iloc[1236, 1])

train_df_copy.iloc[1250, 1] = re.sub(r'#여기', '#Person1#: 여기', train_df_copy.iloc[1250, 1])
train_df_copy.iloc[1266, 1] = re.sub(r'#고객님', '#Person2#: 고객님', train_df_copy.iloc[1266, 1])
train_df_copy.iloc[1278, 1] = re.sub(r'#고객님', '#Person2#: 고객님', train_df_copy.iloc[1278, 1])

train_df_copy.iloc[1281, 1] = re.sub(r'#잠깐만요', '#Person1#: 잠깐만요', train_df_copy.iloc[1281, 1])
train_df_copy.iloc[1283, 1] = re.sub(r'#어디', '#Person1#: 어디', train_df_copy.iloc[1283, 1])
train_df_copy.iloc[1301, 1] = re.sub(r'#샐러드용', '#Person1#: 샐러드용', train_df_copy.iloc[1301, 1])

train_df_copy.iloc[1302, 1] = re.sub(r'#페리에와', '#Person1#: 페리에와', train_df_copy.iloc[1302, 1])
train_df_copy.iloc[1306, 1] = re.sub(r'#나', '#Person2#: 나', train_df_copy.iloc[1306, 1])
train_df_copy.iloc[1322, 1] = re.sub(r'#여기서', '#Person1#: 여기서', train_df_copy.iloc[1322, 1])

train_df_copy.iloc[1547, 1] = re.sub(r'#작은', '#Person2#: 작은', train_df_copy.iloc[1547, 1])
train_df_copy.iloc[1609, 1] = re.sub(r'#여기', '#Person2#: 여기', train_df_copy.iloc[1609, 1])
train_df_copy.iloc[1899, 1] = re.sub(r'#Person#', '#Person2#', train_df_copy.iloc[1899, 1])

train_df_copy.iloc[4537, 1] = re.sub(r'#Person\s', '#Person', train_df_copy.iloc[4537, 1])
train_df_copy.iloc[9750, 1] = re.sub(r'(?<!#)Person', '#Person', train_df_copy.iloc[9750, 1])
train_df_copy.iloc[9779, 1] = re.sub(r'(?<!#)Person', '#Person', train_df_copy.iloc[9779, 1])

In [ ]:
train_df_person.iloc[420, 1] = re.sub(r'#PhoneNumber[가-힣]', '#PhoneNumber#', train_df_person.iloc[420, 1])
train_df_person.iloc[839, 1] = re.sub(r'#사람1', '#Person1#: ', train_df_person.iloc[839, 1])
train_df_person.iloc[1125, 1] = re.sub(r'사람1#', '#Person1#', train_df_person.iloc[1125, 1])

train_df_person.iloc[1142, 1] = re.sub(r'사람1#', '#Person1#', train_df_person.iloc[1142, 1])
train_df_person.iloc[1213, 1] = re.sub(r'#하지만', '#Person1#:', train_df_person.iloc[1213, 1])
train_df_person.iloc[1236, 1] = re.sub(r'#고객님', '#Person2#: 고객님', train_df_person.iloc[1236, 1])

train_df_person.iloc[1250, 1] = re.sub(r'#여기', '#Person1#: 여기', train_df_person.iloc[1250, 1])
train_df_person.iloc[1266, 1] = re.sub(r'#고객님', '#Person2#: 고객님', train_df_person.iloc[1266, 1])
train_df_person.iloc[1278, 1] = re.sub(r'#고객님', '#Person2#: 고객님', train_df_person.iloc[1278, 1])

train_df_person.iloc[1281, 1] = re.sub(r'#잠깐만요', '#Person1#: 잠깐만요', train_df_person.iloc[1281, 1])
train_df_person.iloc[1283, 1] = re.sub(r'#어디', '#Person1#: 어디', train_df_person.iloc[1283, 1])
train_df_person.iloc[1301, 1] = re.sub(r'#샐러드용', '#Person1#: 샐러드용', train_df_person.iloc[1301, 1])

train_df_person.iloc[1302, 1] = re.sub(r'#페리에와', '#Person1#: 페리에와', train_df_person.iloc[1302, 1])
train_df_person.iloc[1306, 1] = re.sub(r'#나', '#Person2#: 나', train_df_person.iloc[1306, 1])
train_df_person.iloc[1322, 1] = re.sub(r'#여기서', '#Person1#: 여기서', train_df_person.iloc[1322, 1])

train_df_person.iloc[1547, 1] = re.sub(r'#작은', '#Person2#: 작은', train_df_person.iloc[1547, 1])
train_df_person.iloc[1609, 1] = re.sub(r'#여기', '#Person2#: 여기', train_df_person.iloc[1609, 1])
train_df_person.iloc[1899, 1] = re.sub(r'#Person#', '#Person2#', train_df_person.iloc[1899, 1])

train_df_person.iloc[4537, 1] = re.sub(r'#Person\s', '#Person', train_df_person.iloc[4537, 1])
train_df_person.iloc[9750, 1] = re.sub(r'(?<!#)Person', '#Person', train_df_person.iloc[9750, 1])
train_df_person.iloc[9779, 1] = re.sub(r'(?<!#)Person', '#Person', train_df_person.iloc[9779, 1])

idx_list, _ = findall_pattern(train_df_person, r'#[가-힣]+')
idx_list2, _ = findall_pattern(train_df_person, r'#\b[A-Za-z가-힣]+\b')
idx_list3, _ = findall_pattern(train_df_person, r'(?<!#)\b[A-Za-z가-힣0-9]+\b#')
idx_list4, _ = findall_pattern(train_df_person, r'(?<!#)\b사람[0-9]+:\s\b') # 사람 Person

person_idx = list(set(idx_list + idx_list2 + idx_list3 + idx_list4))
person_idx = sorted(person_idx)
print(len(person_idx), person_idx)

### 화자가 연속되는 경우

In [ ]:
def reg_masking(pattern, text):
    masked = re.findall(pattern, text)
    return masked

In [ ]:
masked_train = train_df_copy['dialogue'].apply(lambda x: str(reg_masking(r'#\bPerson[0-9]+\b#', x)))
for idx in range(0, train_df_copy.shape[0]):
    print(masked_train[idx])

In [ ]:
def repeat_person(word_list):
    for i in range(len(word_list) - 1):
        if word_list[i] == word_list[i+1]:
            return True
    return False

In [ ]:
masked_train = masked_train.reset_index()
train_df_repeat = masked_train['dialogue'].apply(lambda x: x.lstrip('[').rstrip(']').replace("', '", ' ').strip("'").split())

In [ ]:
idx_repeat = []
for idx in range(0, train_df_copy.shape[0]):
    if repeat_person(train_df_repeat[idx]) == True:
        idx_repeat.append(idx)
print(len(idx_repeat), idx_repeat)

In [ ]:
for idx in idx_repeat:
    print(f'\n[{idx}]')
    print('-----'*20)
    print(train_df_copy.iloc[idx, 1])
    # print(f'>>> {train_df_copy.iloc[idx, 2]}')
    print('-----'*20)

In [ ]:
temp = re.sub(r'않았어요.\r\n#Person1#: ', r'않았어요. ', train_df_copy.iloc[11578, 1])
# temp = re.sub(r'\n#Person1#: 컴퓨터', ' 컴퓨터', temp)
# temp = re.sub(r'#Person2#: 그래', '#Person1#: 그래', temp)
# temp = re.sub(r'#Person1#: 지금', '#Person2#: 지금', temp)
# temp = re.sub(r'#Person2#: 좋아', '#Person1#: 좋아', temp)

print(temp)

In [ ]:
train_df_copy.iloc[345, 1] = re.sub(r'#Person1#: 아니요.', '#Person2#: 아니요.', train_df_copy.iloc[345, 1])
train_df_copy.iloc[484, 1] = re.sub(r'인상적이네.\s', '인상적이네. \n#Person2#: ', train_df_copy.iloc[484, 1])
train_df_copy.iloc[756, 1] = re.sub(r'#Person1#: 오, 안녕, 피비!\n#Person2#:\s', '#Person2#: 오, 안녕, 피비! ', train_df_copy.iloc[756, 1])

train_df_copy.iloc[872, 1] = re.sub(r'#Person1#: 중국은행', '#Person2#: 중국은행', train_df_copy.iloc[872, 1])

temp = re.sub(r'#Person1#: 적자생존', '#Person2#: 적자생존', train_df_copy.iloc[925, 1])
train_df_copy.iloc[925, 1] = re.sub(r'#Person2#: 맞아요', r'#Person1#: 맞아요', temp)

train_df_copy.iloc[982, 1] = re.sub(r'일이죠\?\n#Person2#:\s', '일이죠? ', train_df_copy.iloc[982, 1])

temp = re.sub(r'#Person2#: 아마도', '#Person1#: 아마도', train_df_copy.iloc[1033, 1])
temp = re.sub(r'#Person1#:\s\s', '#Person2#: ', temp)
temp = re.sub(r'#Person2#: 그래', '#Person1#: 그래', temp)
temp = re.sub(r'#Person1#: 지금', '#Person2#: 지금', temp)
train_df_copy.iloc[1033, 1] = re.sub(r'#Person2#: 좋아', '#Person1#: 좋아', temp)

train_df_copy.iloc[1220, 1] = re.sub(r'#Person2#: 음', '#Person1#: 음', train_df_copy.iloc[1220, 1])
train_df_copy.iloc[1294, 1] = re.sub(r'#Person1#: 네,', '#Person2#: 네,', train_df_copy.iloc[1294, 1])

train_df_copy.iloc[1419, 1] = re.sub(r'\n#Person1#: 어떠세요\?', ' 어떠세요?', train_df_copy.iloc[1419, 1])
train_df_copy.iloc[1424, 1] = re.sub(r'\s방으로', '#Person1#: 방으로', train_df_copy.iloc[1424, 1])
train_df_copy.iloc[1440, 1] = re.sub(r'\s언제 예약하시겠습니까\?', '\n#Person1#: 언제 예약하시겠습니까?', train_df_copy.iloc[1440, 1])

temp = re.sub(r'#Person2#: 그거', '#Person1#: 그거', train_df_copy.iloc[1475, 1])
train_df_copy.iloc[1475, 1] = re.sub(r'\n#Person1#: 컴퓨터', ' 컴퓨터', temp)

train_df_copy.iloc[1497, 1] = re.sub(r'\n\s복사', '\n#Person1#: 복사', train_df_copy.iloc[1497, 1])
train_df_copy.iloc[1791, 1] = re.sub(r'#Person1#: 봐', '#Person2#: 봐', train_df_copy.iloc[1791, 1])

train_df_copy.iloc[3628, 1] = re.sub(r'아니라고요. 당신의', '아니라고요.\n#Person1#: 당신의', train_df_copy.iloc[3628, 1])

repl_text = '#Person1#: 안녕, 내가 왔어! 음. . . 너의 머리가 꽤 손상되었고, 끝이 갈라져있어.\n#Person2#: 정말? 어떻게 해야 할까?\n#Person1#: 잘라내면 갈라진 끝은 해결될 거야, 하지만 딥 컨디셔닝 트리트먼트가 필요할 수도 있어.\n#Person2#: 음, 알겠어. 네가 도움이 될 것 같다고 생각하는 대로 해줘.\n#Person1#: 얼굴을 감싸는 레이어도 추가할 거야.\n#Person2#: 위쪽으로 가위로 손을 좀 봐줄 수 있을까? 내 머리 숫이 정말 많아.\n#Person1#: 문제 없어!'
train_df_copy.iloc[5441, 1] = repl_text

train_df_copy.iloc[6759, 1] = re.sub(r'\n#Person2#: 아', '아', train_df_copy.iloc[6759, 1])

train_df_copy.iloc[6799, 1] = re.sub(r'#Person1#: 그,', '#Person2#: 그,', train_df_copy.iloc[6799, 1])
train_df_copy.iloc[8645, 1] = re.sub(r'#Person2#: 저는 서프라이즈', '#Person1#: 저는 서프라이즈', train_df_copy.iloc[8645, 1])
train_df_copy.iloc[9898, 1] = re.sub(r'#Person1#: 오', '#Person2#: 오', train_df_copy.iloc[9898, 1])

train_df_copy.iloc[11578, 1] = re.sub(r'않았어요.\r\n#Person1#: ', r'않았어요. ', train_df_copy.iloc[11578, 1])

In [ ]:
masked_train = train_df_copy['dialogue'].apply(lambda x: str(reg_masking(r'#\bPerson[0-9]+\b#', x)))
for idx in range(0, train_df_copy.shape[0]):
    print(masked_train[idx])

In [ ]:
masked_train = masked_train.reset_index()
train_df_repeat = masked_train['dialogue'].apply(lambda x: x.lstrip('[').rstrip(']').replace("', '", ' ').strip("'").split())

In [ ]:
idx_repeat = []
for idx in range(0, train_df_copy.shape[0]):
    if repeat_person(train_df_repeat[idx]) == True:
        idx_repeat.append(idx)
print(len(idx_repeat), idx_repeat)

## Valid

In [ ]:
dev_df_person = dev_df.copy()
del_token = ['#PhoneNumber#', '#PassportNumber#', '#DateOfBirth#', '#Address#', '#CarNumber#', '#SSN#', '#CardNumber#', '#Email#']
pattern = '|'.join(re.escape(token) for token in del_token)
for idx in tqdm(range(0, dev_df_copy.shape[0])):
    text = dev_df_copy.iloc[idx, 1]
    text2 = re.sub(pattern, '', text)
    dev_df_person.iloc[idx, 1] = text2

### Person이 아니라 한국어인 경우

In [ ]:
idx_list, _ = findall_pattern(dev_df_person, r'#[가-힣]+')
for idx in idx_list:
    print(f'\n[{idx}]')
    print('----'*20)
    print(dev_df_person.iloc[idx, 1])
    print('----'*20)
print(len(idx_list), idx_list)

### 영어와 한국어이면서, 마지막이 숫자가 없거나 #으로 안 끝나는 경우

In [ ]:
idx_list2, _ = findall_pattern(dev_df_person, r'#\b[A-Za-z가-힣]+\b')
for idx in idx_list2:
    print(f'\n[{idx}]')
    print('----'*20)
    print(dev_df_person.iloc[idx, 1])
    print('----'*20)
print(len(idx_list2), idx_list2)

### 첫시작에 #이 없는 경우

In [ ]:
idx_list3, _ = findall_pattern(dev_df_person, r'(?<!#)\b[A-Za-z가-힣0-9]+\b#')
for idx in idx_list3:
    print(f'\n[{idx}]')
    print('----'*20)
    print(dev_df_person.iloc[idx, 1])
    print('----'*20)
print(len(idx_list3))

### 양쪽에 #이 없는 경우

In [ ]:
idx_list4, _ = findall_pattern(dev_df_person, r'(?<!#)\b사람[0-9]+:\s\b') # 사람 Person
for idx in idx_list4:
    print(f'\n[{idx}]')
    print('----'*20)
    print(dev_df_person.iloc[idx, 1])
    print('----'*20)
print(len(idx_list4))

### 4가지 경우의 인덱스 값들 중복 제거

In [ ]:
person_idx = list(set(idx_list + idx_list2 + idx_list3 + idx_list4))
person_idx = sorted(person_idx)
print(len(person_idx), person_idx)

In [ ]:
for idx in person_idx:
    print(f'\n[{idx}]')
    print('----'*20)
    print(dev_df_copy.iloc[idx, 1])
    print('----'*20)

In [ ]:
print(
    re.sub(r'#B형', '#Person1#: B형', dev_df_copy.iloc[63, 1])
)

In [ ]:
dev_df_copy.iloc[48, 1] = re.sub(r'#다음', '#Person1#: 다음', dev_df_copy.iloc[48, 1])
dev_df_copy.iloc[63, 1] = re.sub(r'#B형', '#Person1#: B형', dev_df_copy.iloc[63, 1])

In [ ]:
dev_df_person.iloc[48, 1] = re.sub(r'#다음', '#Person1#: 다음', dev_df_person.iloc[48, 1])
dev_df_person.iloc[63, 1] = re.sub(r'#B형', '#Person1#: B형', dev_df_person.iloc[63, 1])

idx_list, _ = findall_pattern(dev_df_person, r'#[가-힣]+')
idx_list2, _ = findall_pattern(dev_df_person, r'#\b[A-Za-z가-힣]+\b')
idx_list3, _ = findall_pattern(dev_df_person, r'(?<!#)\b[A-Za-z가-힣0-9]+\b#')
idx_list4, _ = findall_pattern(dev_df_person, r'(?<!#)\b사람[0-9]+:\s\b') # 사람 Person

person_idx = list(set(idx_list + idx_list2 + idx_list3 + idx_list4))
person_idx = sorted(person_idx)
print(len(person_idx), person_idx)

### 화자가 연속되는 경우

In [ ]:
def reg_masking(pattern, text):
    masked = re.findall(pattern, text)
    return masked

In [ ]:
masked_train = dev_df_copy['dialogue'].apply(lambda x: str(reg_masking(r'#\bPerson[0-9]+\b#', x)))
for idx in range(0, dev_df_copy.shape[0]):
    print(masked_train[idx])

In [ ]:
def repeat_person(word_list):
    for i in range(len(word_list) - 1):
        if word_list[i] == word_list[i+1]:
            return True
    return False

In [ ]:
masked_train = masked_train.reset_index()
dev_df_repeat = masked_train['dialogue'].apply(lambda x: x.lstrip('[').rstrip(']').replace("', '", ' ').strip("'").split())

In [ ]:
idx_repeat = []
for idx in range(0, dev_df_copy.shape[0]):
    if repeat_person(dev_df_repeat[idx]) == True:
        idx_repeat.append(idx)
print(len(idx_repeat), idx_repeat)

In [ ]:
for idx in idx_repeat:
    print(f'\n[{idx}]')
    print('-----'*20)
    print(dev_df_copy.iloc[idx, 1])
    # print(f'>>> {train_df_copy.iloc[idx, 2]}')
    print('-----'*20)

In [ ]:
temp = re.sub(r'#Person2#: 뭐', r'#Person1#: 뭐', dev_df_copy.iloc[437, 1])
temp = re.sub(r'#Person1#: 봐', r'#Person2#: 봐', temp)
temp = re.sub(r'#Person2#: 그래', r'#Person1#: 그래', temp)
print(temp)

In [ ]:
temp = re.sub(r'#Person2#: 뭐', r'#Person1#: 뭐', dev_df_copy.iloc[437, 1])
temp = re.sub(r'#Person1#: 봐', r'#Person2#: 봐', temp)
dev_df_copy.iloc[437, 1] = re.sub(r'#Person2#: 그래', r'#Person1#: 그래', temp)

In [ ]:
masked_train = dev_df_copy['dialogue'].apply(lambda x: str(reg_masking(r'#\bPerson[0-9]+\b#', x)))
for idx in range(0, dev_df_copy.shape[0]):
    print(masked_train[idx])

In [ ]:
masked_train = masked_train.reset_index()
dev_df_repeat = masked_train['dialogue'].apply(lambda x: x.lstrip('[').rstrip(']').replace("', '", ' ').strip("'").split())

In [ ]:
idx_repeat = []
for idx in range(0, dev_df_copy.shape[0]):
    if repeat_person(dev_df_repeat[idx]) == True:
        idx_repeat.append(idx)
print(len(idx_repeat), idx_repeat)

# special token 재확인

In [ ]:
def reg_masking(text):
    pattern = r'#\w+#'
    masked = re.findall(pattern, text)
    return masked

masked = train_df_copy['dialogue'].apply(lambda x: str(set(reg_masking(x)))) # dev_df_copy
for idx in range(0, train_df_copy.shape[0]): # dev_df_copy
    print(masked[idx])

In [ ]:
temp_df = masked.reset_index()
temp_df['dialogue'] = temp_df['dialogue'].apply(lambda x: set(eval(x)))
unique_dialogues = set().union(*temp_df['dialogue'])
print(sorted(unique_dialogues))

# Basic Preprocessing

In [ ]:
# 좌우 공백 제거
train_df_copy['dialogue'] = train_df_copy['dialogue'].apply(lambda x: x.strip())
train_df_copy['summary'] = train_df_copy['summary'].apply(lambda x: x.strip())
train_df_copy['topic'] = train_df_copy['topic'].apply(lambda x: x.strip())

# 자음, 모음만으로 구성된 경우 적절한 값으로 대체
train_df_copy['dialogue'] = train_df_copy['dialogue'].apply(lambda x: x.replace('제ㅏ', '제가'))
train_df_copy['dialogue'] = train_df_copy['dialogue'].apply(lambda x: x.replace('척했ㄷ거든', '척했거든'))
train_df_copy['dialogue'] = train_df_copy['dialogue'].apply(lambda x: x.replace('배경ㅇ로', '배경으로'))
train_df_copy['dialogue'] = train_df_copy['dialogue'].apply(lambda x: x.replace('ㅋㅋ', '웃기다'))
train_df_copy['dialogue'] = train_df_copy['dialogue'].apply(lambda x: x.replace('아직ㅍ알맞는', '아직 알맞는'))
train_df_copy['summary'] = train_df_copy['summary'].apply(lambda x: x.replace('머라이어 ㅐ리', '머라이어 캐리'))

# 좌우 공백 제거
dev_df_copy['dialogue'] = dev_df_copy['dialogue'].apply(lambda x: x.strip())
dev_df_copy['summary'] = dev_df_copy['summary'].apply(lambda x: x.strip())
dev_df_copy['topic'] = dev_df_copy['topic'].apply(lambda x: x.strip())

In [ ]:
train_df_copy.to_csv('../../data/train_regular-expression.csv', index=False)
dev_df_copy.to_csv('../../data/dev_regular-expression.csv', index=False)

# 대화 중 괄호 안에 있는 문장 제거 / 기타 기호

> token-check 반영한 데이터셋 사용

## Train

In [ ]:
train_df = pd.read_csv(os.path.join(data_path,'train_token-check.csv'))
dev_df = pd.read_csv(os.path.join(data_path,'dev_token-check.csv'))
train_df_copy = train_df.copy()
dev_df_copy = dev_df.copy()

### 괄호 안에 있는 문장 제거

In [ ]:
text = '[네.]]]<<>(네!)[네?]<ab?>'
pattern = r'\[[^\[\]]+\]|\([^\(\)]+\)|\<[^\<\>]+\>'
re.findall(pattern, text)

In [ ]:
temp_list = []
for idx in range(0, train_df_copy.shape[0]):
    text = train_df_copy.iloc[idx, 1]
    temp = re.findall(pattern, text)
    if len(temp) > 0:
        temp_list.append(idx)
print(len(temp_list), temp_list)

In [ ]:
temp_df = train_df_copy.iloc[temp_list]
for i in range(0, temp_df.shape[0]):
    dia = temp_df.iloc[i, 1]
    sum = temp_df.iloc[i, 2]
    print('-----'*20)
    print('[대화]')
    print(dia)
    print('[요약]')
    print(sum)

In [ ]:
train_df_copy['dialogue'] = train_df_copy['dialogue'].apply(lambda x: re.sub(pattern, '', x))
print(temp_df.iloc[0, 1])
print('-----'*20)
print(train_df_copy.iloc[26, 1])

### 기타 기호

In [ ]:
text = '네, 네. 네? 네! 네~ "네"' # '
pattern = r'[,~"]'
re.findall(pattern, text)

In [ ]:
temp_list = []
for idx in range(0, train_df_copy.shape[0]):
    text = train_df_copy.iloc[idx, 1]
    temp = re.findall(pattern, text)
    if len(temp) > 0:
        temp_list.append(idx)
print(len(temp_list), temp_list)

In [ ]:
temp_df = train_df_copy.iloc[temp_list]

In [ ]:
train_df_copy['dialogue'] = train_df_copy['dialogue'].apply(lambda x: re.sub(pattern, '', x))
print(temp_df.iloc[0, 1])
print('-----'*20)
print(train_df_copy.iloc[0, 1])

In [ ]:
text = "'네'"
pattern = r"\'"
re.findall(pattern, text)

In [ ]:
temp_list = []
for idx in range(0, train_df_copy.shape[0]):
    text = train_df_copy.iloc[idx, 1]
    temp = re.findall(pattern, text)
    if len(temp) > 0:
        temp_list.append(idx)
print(len(temp_list), temp_list)

In [ ]:
temp_df = train_df_copy.iloc[temp_list]

In [ ]:
train_df_copy['dialogue'] = train_df_copy['dialogue'].apply(lambda x: re.sub(pattern, '', x))
print(temp_df.iloc[0, 1])
print('-----'*20)
print(train_df_copy.iloc[33, 1])

## Valid

### 괄호 안에 있는 문장 제거

In [ ]:
text = '[네.]]]<<>(네!)[네?]<ab?>'
pattern = r'\[[^\[\]]+\]|\([^\(\)]+\)|\<[^\<\>]+\>'
re.findall(pattern, text)

In [ ]:
temp_list = []
for idx in range(0, dev_df_copy.shape[0]):
    text = dev_df_copy.iloc[idx, 1]
    temp = re.findall(pattern, text)
    if len(temp) > 0:
        temp_list.append(idx)
print(len(temp_list), temp_list)

In [ ]:
temp_df = dev_df_copy.iloc[temp_list]
for i in range(0, temp_df.shape[0]):
    dia = temp_df.iloc[i, 1]
    sum = temp_df.iloc[i, 2]
    print('-----'*20)
    print('[대화]')
    print(dia)
    print('[요약]')
    print(sum)

In [ ]:
dev_df_copy['dialogue'] = dev_df_copy['dialogue'].apply(lambda x: re.sub(pattern, '', x))
print(temp_df.iloc[0, 1])
print('-----'*20)
print(dev_df_copy.iloc[38, 1])

### 기타 기호

In [ ]:
text = '네, 네. 네? 네! 네~ "네"'
pattern = r'[,~"]'
re.findall(pattern, text)

In [ ]:
temp_list = []
for idx in range(0, dev_df_copy.shape[0]):
    text = dev_df_copy.iloc[idx, 1]
    temp = re.findall(pattern, text)
    if len(temp) > 0:
        temp_list.append(idx)
print(len(temp_list), temp_list)

In [ ]:
temp_df = dev_df_copy.iloc[temp_list]

In [ ]:
dev_df_copy['dialogue'] = dev_df_copy['dialogue'].apply(lambda x: re.sub(pattern, '', x))
print(temp_df.iloc[0, 1])
print('-----'*20)
print(dev_df_copy.iloc[0, 1])

In [ ]:
text = "'네'"
pattern = r"\'"
re.findall(pattern, text)

In [ ]:
temp_list = []
for idx in range(0, dev_df_copy.shape[0]):
    text = dev_df_copy.iloc[idx, 1]
    temp = re.findall(pattern, text)
    if len(temp) > 0:
        temp_list.append(idx)
print(len(temp_list), temp_list)

In [ ]:
temp_df = dev_df_copy.iloc[temp_list]

In [ ]:
dev_df_copy['dialogue'] = dev_df_copy['dialogue'].apply(lambda x: re.sub(pattern, '', x))
print(temp_df.iloc[0, 1])
print('-----'*20)
print(dev_df_copy.iloc[54, 1])

- 알겠습니다, 감사합니다, 네 같은거도 지우려고 했으나 화자의 문장 전체가 사라지는 경우가 있어 적용하지 않음

In [ ]:
train_df_copy.to_csv('../../data/train_regular-expression2.csv', index=False)
dev_df_copy.to_csv('../../data/dev_regular-expression2.csv', index=False)